# **Goals**
##In this section, our focus was on preparing the data for a hierarchical model intended to cluster patient samples based on shared characteristics. The primary objective was to uncover patterns and similarities without depending on predefined rules that typically serve as ground truth for predictive models since such rules were not consistently available.




In [ ]:
!pip install pandas
import numpy as np
from scipy.stats import skewnorm
import matplotlib.pyplot as plt
import pandas as pd
from scipy import linspace
from scipy import pi,sqrt,exp
from scipy.special import erf
from pylab import plot,show
import seaborn as sns

In [ ]:
!pip install google.colab
from google.colab import drive
drive.mount('/content/d')

In [ ]:
# #Linoy's file path
file_path = '/content/d/MyDrive/final_project/Avera_Atlantic_Data.csv'
df = pd.read_csv(file_path)

In [ ]:
# # Sahar file path
# file_path = '/content/d/MyDrive/Avera_Atlantic_Data.csv'
# df = pd.read_csv(file_path, encoding='utf-8-sig')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# **Data Preperation for using a hirarchical model**

In [ ]:
df['Patient Class'].value_counts()

In [ ]:
#Dropping the target columns and the non numeric column- Exam Description
df_copy = df
df_copy = df_copy.drop(['Exam Description', 'Avera_Urgency', 'Atlantic_Urgency', 'Average_Urgency'], axis=1)

In [ ]:
df_copy

### **Encoding**

In [ ]:
# Encoding the df_copy to numeric dataframe
from sklearn.preprocessing import LabelEncoder

# Create an instance of LabelEncoder
encoder = LabelEncoder()

# Create an empty dictionary to store the reverse mapping
reverse_mapping = {}

# Iterate through each column and encode the categorical data
for column in ['Tenant DESC', 'Priority', 'Patient Class', 'Modality', 'Body Part', 'Subspecialty']:
    encoded_values = encoder.fit_transform(df_copy[column])
    df_copy[column] = encoded_values

    # Create the reverse mapping for the column
    reverse_mapping[column] = {i: category for i, category in enumerate(encoder.classes_)}

# Print the encoded dataframe
df_copy

In [ ]:
#Checking for null values
df_copy.isna().value_counts()

# **Hierarchical model**

In [ ]:
# This code use the df_copy and one of the methods to calculate the linkage matrix.
# Then it applies hierarchical clustering to predict clusters based on a distance threshold,
# which is set to the maximum distance between clusters corresponding to 10-15 clusters.
# The dendrogram is plotted with truncated branches using truncate_mode='lastp' and p=100,
# which means only the last 100 merged clusters are shown.
# Show_leaf_counts=True displays the number of samples in each leaf node.

import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram, fcluster, linkage

def hierarchical_clustering(df_copy, method, max_d, Cluster_Method):
    # Calculate linkage matrix using the specified method
    Z = linkage(df_copy, method=method)

    # Calculate the maximum distance between clusters corresponding to the specified number of clusters
    max_dist = Z[-max_d, 2]

    # Predict clusters based on distance threshold
    clusters = fcluster(Z, t=max_dist, criterion='distance')

    # Add 'Cluster_Ward' column to the original dataframe
    df_copy[Cluster_Method] = clusters

    # Plot dendrogram with truncated branches
    plt.figure(figsize=(15, 10))
    plt.title('Hierarchical Clustering Dendrogram Of ' + Cluster_Method )
    plt.xlabel('sample index')
    plt.ylabel('distance')
    dendrogram(Z, truncate_mode='lastp', p=100, show_leaf_counts=True, leaf_rotation=90., leaf_font_size=8.)
    plt.show()


In [ ]:
#WARD
hierarchical_clustering(df_copy, method="ward", max_d=50, Cluster_Method='Cluster_Ward')

In [ ]:
df_copy

In [ ]:
#SINGLE
hierarchical_clustering(df_copy, method="single", max_d=40, Cluster_Method='Cluster_Single')

In [ ]:
df_copy

In [ ]:
#AVERAGE
hierarchical_clustering(df_copy, method="average", max_d=50, Cluster_Method='Cluster_Average')

In [ ]:
df_copy

In [ ]:
#COMPLETE
hierarchical_clustering(df_copy, method="complete", max_d=45, Cluster_Method='Cluster_Complete')

In [ ]:
df_copy

### **Decoding**

In [ ]:
#Creating a new df with the urgency and cluster columns for the regression and clasification models
dfs_to_concat = [df_copy, df['Avera_Urgency'], df['Atlantic_Urgency'], df['Average_Urgency']]
concatenated_df = pd.concat(dfs_to_concat, axis=1)
concatenated_df

concatenated_df.to_csv('data_for_regression_model.csv')

# Iterate through each column in the encoded DataFrame
for column in df_copy.columns:
    if column in reverse_mapping:
        # Retrieve the reverse mapping dictionary for the column
        mapping = reverse_mapping[column]

        # Decode the encoded values using the reverse mapping dictionary
        decoded_values = df_copy[column].map(mapping)

        # Replace the values in the corresponding column of df_copy with the decoded values
        df_copy[column] = decoded_values

# Print the modified df_copy DataFrame
df_copy

In [ ]:
# Export an Excel file to present the results of each method
# Print each method in a different file
import pandas as pd

def create_file_for_method(df_copy, range_value, cluster_column, output_file='cluster_results.xlsx', df_results=None):
    # Create an empty DataFrame to store the cluster results if df_results is not provided
    if df_results is None:
        df_results = pd.DataFrame()

    # Iterate through each cluster
    for cluster in range(*range_value):
        cluster_data = df_copy[df_copy[cluster_column] == cluster]

        # Print the number of samples in the current cluster
        num_samples = len(cluster_data)

        # Create a dictionary to store the cluster results
        cluster_result = {cluster_column: cluster, 'Number of samples': num_samples}

        # Iterate through each selected column and print unique string values
        for column in ['Tenant DESC', 'Priority', 'Patient Class', 'Modality', 'Body Part', 'Subspecialty']:
            unique_values = cluster_data[column].unique()
            unique_values = [str(val) for val in unique_values]  # Convert to strings
            cluster_result[column] = ', '.join(unique_values)

        df_results = df_results.append(cluster_result, ignore_index=True)

    # Save the results to an Excel file
    df_results.to_excel(output_file, index=False)

    return df_results


# **Visualizations**

## **Cluster Ward**

In [ ]:
# Using the transform function with WARD method and 30 clusters
df_ward_results=pd.DataFrame()
df_ward_results=create_file_for_method(df_copy, range_value=(1, 51), cluster_column='Cluster_Ward',output_file='cluster_ward_results.xlsx', df_results=df_ward_results)


In [ ]:
# Creating a matrix to visualize the number of each modality in the cluster 'Ward'
cluster_modality_ward = df_copy.groupby('Cluster_Ward')['Modality'].value_counts(normalize=True).unstack().fillna(0)
cluster_modality_ward

In [ ]:
# Visualize the results in heatmap
sns.heatmap(cluster_modality_ward)
plt.title("Correlations between modality and cluster Ward")

In [ ]:
# Normalized distribution of modality by cluster Ward
cluster_modality_ward.loc['all'] = df_copy['Modality'].value_counts(normalize=True).fillna(0)
cluster_modality_ward.plot(kind='bar', figsize=(9,3), stacked=True, cmap='tab20b').legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.title("Normalized distribution of modality by cluster Ward")
plt.xlabel("Cluster Ward")
plt.ylabel("%")

In [ ]:
# Creating a matrix to visualize the number of each patient class in the cluster 'Ward'
cluster_pc_ward = df_copy.groupby('Cluster_Ward')['Patient Class'].value_counts(normalize=True).unstack().fillna(0)
cluster_pc_ward

In [ ]:
# Visualize the results in barplot
cluster_pc_ward.loc['all'] = df_copy['Patient Class'].value_counts(normalize=True).fillna(0)
cluster_pc_ward.plot(kind='bar', figsize=(9,3), stacked=True, cmap='tab20b').legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.title("Normalized distribution of patient class by cluster Ward")
plt.xlabel("Cluster Ward")
plt.ylabel("%")

In [ ]:
# Visualize the results in heatmap
sns.heatmap(cluster_pc_ward)
plt.title("Correlations between patient class and cluster Ward")

In [ ]:
# Creating a matrix to visualize the number of each body part in the cluster 'Ward'
cluster_bd_ward = df_copy.groupby('Cluster_Ward')['Body Part'].value_counts(normalize=True).unstack().fillna(0)
cluster_bd_ward

In [ ]:
# Visualize the results in heatmap
sns.heatmap(cluster_bd_ward)
plt.title("Correlations between of body part and cluster Ward")

In [ ]:
# Creating a matrix to visualize the number of each subspecialty in the cluster 'Ward'
cluster_subs_ward=df_copy.groupby('Cluster_Ward')['Subspecialty'].value_counts(normalize=True).unstack().fillna(0)
cluster_subs_ward

In [ ]:
# Visualize the results in heatmap
sns.heatmap(cluster_subs_ward)
plt.title("Correlations between of subspecialty and cluster Ward")

In [ ]:
# Visualize the results in barplot
cluster_subs_ward.loc['all'] = df_copy['Subspecialty'].value_counts(normalize=True).fillna(0)
cluster_subs_ward.plot(kind='bar', figsize=(9,5), stacked=True, cmap='tab20b').legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.title('Normalized distribution of subspecialty by cluster Ward')
plt.xlabel("Cluster_ward")
plt.ylabel("%")

In [ ]:
# Creating a matrix to visualize the number of each Priority in the cluster 'Ward'
cluster_P_ward = df_copy.groupby('Cluster_Ward')['Priority'].value_counts(normalize=True).unstack().fillna(0)
cluster_P_ward

In [ ]:
# Visualize the results in heatmap
sns.heatmap(cluster_P_ward)
plt.title("Correlations between priority and cluster Ward")

In [ ]:
# Visualize the results in barplot
cluster_P_ward.loc['all'] = df_copy['Priority'].value_counts(normalize=True).fillna(0)
cluster_P_ward.plot(kind='bar', figsize=(9,3), stacked=True, cmap='tab20b').legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.title("Normelized distribution of priority by cluster Ward")
plt.xlabel("Cluster Ward")
plt.ylabel("%")

## **Cluster Single**

In [ ]:
# Using the transform function with SINGLE method and 40 clusters
df_single_results=pd.DataFrame()
df_single_results=create_file_for_method(df_copy, range_value=(1, 41), cluster_column='Cluster_Single', output_file='cluster_single_results.xlsx', df_results=df_single_results)

In [ ]:
# Creating a matrix to visualize the number of each modality in the cluster 'Single'
cluster_modality_single=df_copy.groupby('Cluster_Single')['Modality'].value_counts(normalize=True).unstack().fillna(0)
cluster_modality_single

In [ ]:
# Visualize the results in heatmap
sns.heatmap(cluster_modality_single)
plt.title("Correlations between modality and cluster Ward")

In [ ]:
# Normalized distribution of modality by cluster Single
cluster_modality_single.loc['all'] = df_copy['Modality'].value_counts(normalize=True).fillna(0)
cluster_modality_single.plot(kind='bar', figsize=(9,3), stacked=True, cmap='tab20b').legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.title("Normalized distribution of modality by cluster Single")
plt.xlabel("Cluster Single")
plt.ylabel("%")

In [ ]:
# Creating a matrix to visualize the number of each patient class in the cluster 'Single'
cluster_pc_single = df_copy.groupby('Cluster_Single')['Patient Class'].value_counts(normalize=True).unstack().fillna(0)
cluster_pc_single

In [ ]:
# Visualize the results in barplot
cluster_pc_single.loc['all'] = df_copy['Patient Class'].value_counts(normalize=True).fillna(0)
cluster_pc_single.plot(kind='bar', figsize=(9,3), stacked=True, cmap='tab20b').legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.title("Normalized distribution of patient class by cluster Single")
plt.xlabel("Cluster Single")
plt.ylabel("%")

In [ ]:
# Visualize the results in heatmap
sns.heatmap(cluster_pc_single)
plt.title("Correlations between patient class and cluster Single")

In [ ]:
# Creating a matrix to visualize the number of each modality in the cluster 'Single'
cluster_bp_single=df_copy.groupby('Cluster_Single')['Body Part'].value_counts(normalize=True).unstack().fillna(0)
cluster_bp_single

In [ ]:
# Visualize the results in heatmap
sns.heatmap(cluster_bp_single)
plt.title("Correlations between body part and cluster Single")

In [ ]:
# Creating a matrix to visualize the number of each subspecialty in the cluster 'Single'
cluster_subs_single=df_copy.groupby('Cluster_Single')['Subspecialty'].value_counts(normalize=True).unstack().fillna(0)
cluster_subs_single

In [ ]:
# Visualize the results in heatmap
sns.heatmap(cluster_subs_single)
plt.title("Correlations between subspecialty and cluster Single")

In [ ]:
# Visualize the results in barplot
cluster_subs_single.loc['all'] = df_copy['Subspecialty'].value_counts(normalize=True).fillna(0)
cluster_subs_single.plot(kind='bar', figsize=(9,5), stacked=True, cmap='tab20b').legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.title('Normalized distribution of subspecialty by cluster Single')
plt.xlabel("Cluster_single")
plt.ylabel("%")

In [ ]:
# Creating a matrix to visualize the number of each Priority in the cluster 'single'
cluster_P_single = df_copy.groupby('Cluster_Single')['Priority'].value_counts(normalize=True).unstack().fillna(0)
cluster_P_single

In [ ]:
# Visualize the results in heatmap
sns.heatmap(cluster_P_single)
plt.title("Correlations between priority and cluster Single")

In [ ]:
# Visualize the results in barplot
cluster_P_single.loc['all'] = df_copy['Priority'].value_counts(normalize=True).fillna(0)
cluster_P_single.plot(kind='bar', figsize=(9,3), stacked=True, cmap='tab20b').legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.title("Normelized distribution of priority by cluster Single")
plt.xlabel("Cluster Single")
plt.ylabel("%")

## **Cluster Average**

In [ ]:
# Using the transform function with AVEREGE method and 28 clusters
df_average_results=pd.DataFrame()
df_average_results=create_file_for_method(df_copy, range_value=(1, 51), cluster_column='Cluster_Average', output_file='cluster_average_results.xlsx', df_results=df_average_results)

In [ ]:
# Creating a matrix to visualize the number of each modality in the cluster 'Average'
cluster_modality_average=df_copy.groupby('Cluster_Average')['Modality'].value_counts(normalize=True).unstack().fillna(0)
cluster_modality_average

In [ ]:
# Visualize the results in heatmap
sns.heatmap(cluster_modality_average)
plt.title("Correlations between modality and cluster Average")

In [ ]:
# Normalized distribution of modality by cluster Average
cluster_modality_average.loc['all'] = df_copy['Modality'].value_counts(normalize=True).fillna(0)
cluster_modality_average.plot(kind='bar', figsize=(9,3), stacked=True, cmap='tab20b').legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.title("Normalized distribution of modality by cluster Average")
plt.xlabel("Cluster Average")
plt.ylabel("%")

In [ ]:
# Creating a matrix to visualize the number of each patient class in the cluster 'Average'
cluster_pc_average = df_copy.groupby('Cluster_Average')['Patient Class'].value_counts(normalize=True).unstack().fillna(0)
cluster_pc_average

In [ ]:
# Visualize the results in barplot
cluster_pc_average.loc['all'] = df_copy['Patient Class'].value_counts(normalize=True).fillna(0)
cluster_pc_average.plot(kind='bar', figsize=(9,3), stacked=True, cmap='tab20b').legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.title('Normalized distribution of patient class by cluster Average')
plt.xlabel("Cluster_average")
plt.ylabel("%")

In [ ]:
# Visualize the results in heatmap
sns.heatmap(cluster_pc_average)
plt.title("Correlations between patient class and cluster Average")

In [ ]:
# Creating a matrix to visualize the number of each body part in the cluster 'Average'
cluster_bd_average = df_copy.groupby('Cluster_Average')['Body Part'].value_counts().unstack().fillna(0)
cluster_bd_average

In [ ]:
# Visualize the results in heatmap
sns.heatmap(cluster_bd_average)
plt.title("Correlations between body part and cluster Average")

In [ ]:
# Creating a matrix to visualize the number of each subspecialty in the cluster 'Average'
cluster_subs_average=df_copy.groupby('Cluster_Average')['Subspecialty'].value_counts(normalize=True).unstack().fillna(0)
cluster_subs_average

In [ ]:
# Visualize the results in heatmap
sns.heatmap(cluster_subs_average)
plt.title("Correlations between subspecialty and cluster Average")

In [ ]:
# Visualize the results in barplot
cluster_subs_average.loc['all'] = df_copy['Subspecialty'].value_counts(normalize=True).fillna(0)
cluster_subs_average.plot(kind='bar', figsize=(9,5), stacked=True, cmap='tab20b').legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.title('Normalized distribution of subspecialty by cluster Average')
plt.xlabel("Cluster_average")
plt.ylabel("%")

In [ ]:
# Creating a matrix to visualize the number of each Priority in the cluster 'average'
cluster_P_average = df_copy.groupby('Cluster_Average')['Priority'].value_counts(normalize=True).unstack().fillna(0)
cluster_P_average

In [ ]:
# Visualize the results in heatmap
sns.heatmap(cluster_P_average)
plt.title("Correlations between priority and cluster Average")

In [ ]:
# Visualize the results in barplot
cluster_P_average.loc['all'] = df_copy['Priority'].value_counts(normalize=True).fillna(0)
cluster_P_average.plot(kind='bar', figsize=(9,3), stacked=True, cmap='tab20b').legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.title("Normelized distribution of priority by cluster Average")
plt.xlabel("Cluster Average")
plt.ylabel("%")

## **Cluster Complete**

In [ ]:
# Using the transform function with COMPLETE method and 45 clusters
df_complete_results=pd.DataFrame()
df_complete_results=create_file_for_method(df_copy, range_value=(1, 46), cluster_column='Cluster_Complete', output_file='cluster_complete_results.xlsx', df_results=df_complete_results)

In [ ]:
# Creating a matrix to visualize the number of each modality in the cluster 'Complete'
cluster_modality_complete=df_copy.groupby('Cluster_Complete')['Modality'].value_counts(normalize=True).unstack().fillna(0)
cluster_modality_complete

In [ ]:
# Visualize the results in heatmap
sns.heatmap(cluster_modality_complete)
plt.title("Correlations between modality and cluster Complete")

In [ ]:
# Normalized distribution of modality by cluster Complete
cluster_modality_complete.loc['all'] = df_copy['Modality'].value_counts(normalize=True).fillna(0)
cluster_modality_complete.plot(kind='bar', figsize=(9,3), stacked=True, cmap='tab20b').legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.title("Normalized distribution of modality by cluster Complete")
plt.xlabel("Cluster Complete")
plt.ylabel("%")

In [ ]:
# Creating a matrix to visualize the number of each patient class in the cluster 'Complete''
cluster_pc_complete = df_copy.groupby('Cluster_Complete')['Patient Class'].value_counts(normalize=True).unstack().fillna(0)
cluster_pc_complete

In [ ]:
# Visualize the results in barplot
cluster_pc_complete.loc['all'] = df_copy['Patient Class'].value_counts(normalize=True).fillna(0)
cluster_pc_complete.plot(kind='bar', figsize=(9,3), stacked=True, cmap='tab20b').legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.title("Normalized distribution of patient class by cluster Complete")
plt.xlabel("Cluster Complete")
plt.ylabel("%")

In [ ]:
# Visualize the results in heatmap
sns.heatmap(cluster_pc_complete)
plt.title("Correlations between patient class and cluster Complete")

In [ ]:
# Creating a matrix to visualize the number of each body part in the cluster 'Complete'
cluster_bd_complete = df_copy.groupby('Cluster_Complete')['Body Part'].value_counts(normalize=True).unstack().fillna(0)
cluster_bd_complete

In [ ]:
# Visualize the results in heatmap
sns.heatmap(cluster_bd_complete)
plt.title("Correlations between body part and cluster Complete")

In [ ]:
# Creating a matrix to visualize the number of each subspecialty in the cluster 'Complete'
cluster_subs_complete = df_copy.groupby('Cluster_Complete')['Subspecialty'].value_counts(normalize=True).unstack().fillna(0)
cluster_subs_complete

In [ ]:
# Visualize the results in heatmap
sns.heatmap(cluster_subs_complete)
plt.title("Correlations between subspecialty and cluster Complete")

In [ ]:
# Visualize the results in barplot
cluster_subs_complete.loc['all'] = df_copy['Subspecialty'].value_counts(normalize=True).fillna(0)
cluster_subs_complete.plot(kind='bar', figsize=(9,5), stacked=True, cmap='tab20b').legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.title('Normalized distribution of subspecialty by cluster Complete')
plt.xlabel("Cluster_complete")
plt.ylabel("%")

In [ ]:
# Creating a matrix to visualize the number of each Priority in the cluster 'complete'
cluster_P_complete = df_copy.groupby('Cluster_Complete')['Priority'].value_counts(normalize=True).unstack().fillna(0)
cluster_P_complete

In [ ]:
# Visualize the results in heatmap
sns.heatmap(cluster_P_complete)
plt.title("Correlations between priority and cluster Complete")

In [ ]:
# Visualize the results in barplot
cluster_P_complete.loc['all'] = df_copy['Priority'].value_counts(normalize=True).fillna(0)
cluster_P_complete.plot(kind='bar', figsize=(9,3), stacked=True, cmap='tab20b').legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.title("Normelized distribution of priority by cluster Complete")
plt.xlabel("Cluster Complete")
plt.ylabel("%")

# **Adding the target variable (urgency score)**

In [ ]:
# Adding the urgency columns to the df
df_copy = pd.concat([df_copy, df['Atlantic_Urgency'], df['Avera_Urgency'], df['Average_Urgency']], axis=1)
df_copy
df_copy.to_csv('data_for_regression_model.csv')

In [ ]:
# A function to plot the urgencies in a boxplot by cluster, method, and Tenant
def plot_urgencies_by_cluster(df_copy, method, Tenant):
    # Group the data by the cluster and select the Tenant
    grouped_data = df_copy.groupby(method)[Tenant]

    # Collect the data for each cluster as a list
    data_by_cluster = [grouped_data.get_group(cluster).values for cluster in grouped_data.groups]

    # Set the figure size
    plt.figure(figsize=(10, 6))  # Adjust the values (width, height) as desired

    # Create the box plot
    plt.boxplot(data_by_cluster, patch_artist=True)

    # Customize the plot
    plt.title(f"Distribution of {Tenant} by {method}")
    plt.xlabel(f"{method}")
    plt.ylabel(f"{Tenant}")

    # Set the x-axis tick labels to match the cluster names
    plt.xticks(range(1, len(data_by_cluster) + 1), grouped_data.groups)

    # Display the plot
    plt.show()

# **Visualizations with urgency score**

## **Cluster Ward**

In [ ]:
# Creating a matrix to visualize the number of each urgency in the Tenant Atlantic in the cluster 'Ward'
ward_atlantic_urgency= df_copy.groupby('Cluster_Ward')['Atlantic_Urgency'].value_counts(normalize=True).unstack().fillna(0)
ward_atlantic_urgency

In [ ]:
# Visualize the results in heatmap
sns.heatmap(ward_atlantic_urgency)
plt.title("Correlations between Atlantic urgency and cluster Ward")

In [ ]:
# Use the function to create a boxplot with the disterbution
plot_urgencies_by_cluster(df_copy, 'Cluster_Ward', 'Atlantic_Urgency')

In [ ]:
# Normalized distribution of Atlantic urgency by ward cluster
ward_atlantic_urgency.loc['all'] = df_copy['Atlantic_Urgency'].value_counts(normalize=True).fillna(0)
ax = ward_atlantic_urgency.plot(kind='bar', figsize=(9, 3), stacked=True, cmap='tab20b')
plt.title("Normalized distribution of Atlantic urgency by cluster Ward")
plt.ylabel('%')
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

In [ ]:
# Creating a matrix to visualize the number of each urgency in the Tenant Avera in the cluster 'Ward'
ward_avera_urgency= df_copy.groupby('Cluster_Ward')['Avera_Urgency'].value_counts(normalize=True).unstack().fillna(0)
ward_avera_urgency

In [ ]:
# Visualize the results in heatmap
sns.heatmap(ward_avera_urgency)
plt.title("Correlations between Avera urgency and cluster Ward")

In [ ]:
# Use the function to create a boxplot with the disterbution of Avera's urgencies by ward cluster
plot_urgencies_by_cluster(df_copy, 'Cluster_Ward', 'Avera_Urgency')

In [ ]:
# Normalized distribution of Avera urgency by Ward cluster
ward_avera_urgency.loc['all'] = df_copy['Avera_Urgency'].value_counts(normalize=True).fillna(0)
ax = ward_avera_urgency.plot(kind='bar', figsize=(9, 3), stacked=True, cmap='tab20b')
plt.title("Normalized distribution of Avera urgency by cluster Ward")
plt.ylabel('%')
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

## **Cluster Single**

In [ ]:
# Creating a matrix to visualize the number of each urgency in the Tenant Atlantic in the cluster 'Single'
single_atlantic_urgency= df_copy.groupby('Cluster_Single')['Atlantic_Urgency'].value_counts(normalize=True).unstack().fillna(0)
single_atlantic_urgency

In [ ]:
# Visualize the results in heatmap
sns.heatmap(single_atlantic_urgency)
plt.title("Correlations between Atlantic urgency and cluster Single")

In [ ]:
# Use the function to create a boxplot with the disterbution of Atlantic urgency by Single cluster
plot_urgencies_by_cluster(df_copy, 'Cluster_Single', 'Atlantic_Urgency')

In [ ]:
# Normalized distribution of Atlantic urgency by Single cluster
single_atlantic_urgency.loc['all'] = df_copy['Atlantic_Urgency'].value_counts(normalize=True).fillna(0)
ax = single_atlantic_urgency.plot(kind='bar', figsize=(9, 3), stacked=True, cmap='tab20b')
plt.title("Normalized distribution of Atlantic urgency by cluster Single")
plt.ylabel('%')
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

In [ ]:
# Creating a matrix to visualize the number of each urgency in the Tenant Avera in the cluster 'Single'
single_avera_urgency= df_copy.groupby('Cluster_Single')['Avera_Urgency'].value_counts(normalize=True).unstack().fillna(0)
single_avera_urgency

In [ ]:
# Visualize the results in heatmap
sns.heatmap(single_avera_urgency)
plt.title("Correlations between Avera urgency and cluster Single")

In [ ]:
# Use the function to create a boxplot with the disterbution of Avera urgency by cluster Single
plot_urgencies_by_cluster(df_copy, 'Cluster_Single', 'Avera_Urgency')

In [ ]:
# Normalized distribution of Avera urgency by Single cluster
single_avera_urgency.loc['all'] = df_copy['Avera_Urgency'].value_counts(normalize=True).fillna(0)
ax = single_avera_urgency.plot(kind='bar', figsize=(9, 3), stacked=True, cmap='tab20b')
plt.title("Normalized distribution of Avera urgency by cluster Single")
plt.ylabel('%')
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

## **Cluster Average**

In [ ]:
# Creating a matrix to visualize the number of each urgency in the Tenant Avera in the cluster 'Average'
average_atlantic_urgency= df_copy.groupby('Cluster_Average')['Atlantic_Urgency'].value_counts(normalize=True).unstack().fillna(0)
average_atlantic_urgency

In [ ]:
# Visualize the results in heatmap
sns.heatmap(average_atlantic_urgency)
plt.title("Correlations between Atlantic urgency and cluster Average")

In [ ]:
# Use the function to create a boxplot with the disterbution of Atlantic urgency by cluster Average
plot_urgencies_by_cluster(df_copy, 'Cluster_Average', 'Atlantic_Urgency')

In [ ]:
# Normalized distribution of Atlantic urgency by Average cluster
average_atlantic_urgency.loc['all'] = df_copy['Atlantic_Urgency'].value_counts(normalize=True).fillna(0)
ax = average_atlantic_urgency.plot(kind='bar', figsize=(9, 3), stacked=True, cmap='tab20b')
plt.title("Normalized distribution of Atlantic urgency by cluster Average")
plt.ylabel('%')
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

In [ ]:
# Creating a matrix to visualize the number of each urgency in the Tenant Avera in the cluster 'Average'
average_avera_urgency= df_copy.groupby('Cluster_Average')['Avera_Urgency'].value_counts(normalize=True).unstack().fillna(0)
average_avera_urgency

In [ ]:
# Visualize the results in heatmap
sns.heatmap(average_avera_urgency)
plt.title("Correlations between Avera urgency and cluster Average")

In [ ]:
# Use the function to create a boxplot with the disterbution of Avera urgency by cluster Average
plot_urgencies_by_cluster(df_copy, 'Cluster_Average', 'Avera_Urgency')

In [ ]:
# Normalized distribution of Avera urgency by Average cluster
average_avera_urgency.loc['all'] = df_copy['Avera_Urgency'].value_counts(normalize=True).fillna(0)
ax = average_avera_urgency.plot(kind='bar', figsize=(9, 3), stacked=True, cmap='tab20b')
plt.title("Normalized distribution of Avera urgency by cluster Average")
plt.ylabel('%')
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

## **Cluster Complete**

In [ ]:
# Creating a matrix to visualize the number of each urgency in the Tenant Atlantic in the cluster 'Complete'
complete_atlantic_urgency= df_copy.groupby('Cluster_Complete')['Atlantic_Urgency'].value_counts(normalize=True).unstack().fillna(0)
complete_atlantic_urgency

In [ ]:
# Visualize the results in heatmap
sns.heatmap(complete_atlantic_urgency)
plt.title("Correlations between Atlantic urgency and cluster Complete")

In [ ]:
# Use the function to create a boxplot with the disterbution of Atlantic urgency by cluster Complete
plot_urgencies_by_cluster(df_copy, 'Cluster_Complete', 'Atlantic_Urgency')

In [ ]:
# Normalized distribution of Atlantic urgency by Complete cluster
complete_atlantic_urgency.loc['all'] = df_copy['Atlantic_Urgency'].value_counts(normalize=True).fillna(0)
ax = complete_atlantic_urgency.plot(kind='bar', figsize=(9, 3), stacked=True, cmap='tab20b')
plt.title("Normalized distribution of Atlantic urgency by cluster Complete")
plt.ylabel('%')
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

In [ ]:
# Creating a matrix to visualize the number of each urgency in the Tenant Avera in the cluster 'Complete'
complete_avera_urgency= df_copy.groupby('Cluster_Complete')['Avera_Urgency'].value_counts(normalize=True).unstack().fillna(0)
complete_avera_urgency

In [ ]:
# Visualize the results in heatmap
sns.heatmap(complete_avera_urgency)
plt.title("Correlations between Avera urgency and cluster Complete")

In [ ]:
# Use the function to create a boxplot with the disterbution
plot_urgencies_by_cluster(df_copy, 'Cluster_Complete', 'Avera_Urgency')

In [ ]:
# Normalized distribution of Avera urgency by Complete cluster
complete_avera_urgency.loc['all'] = df_copy['Avera_Urgency'].value_counts(normalize=True).fillna(0)
ax = complete_avera_urgency.plot(kind='bar', figsize=(9, 3), stacked=True, cmap='tab20b')
plt.title("Normalized distribution of Avera urgency by cluster Complete")
plt.ylabel('%')
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

In [ ]:
df_copy.to_csv('data_for_regression_model_textual.csv')